In [2]:
%classpath add mvn org.apache.spark spark-sql_2.11 2.1.0
org.apache.log4j.Logger.getRootLogger().setLevel(org.apache.log4j.Level.ERROR);


null

In [3]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._ 


import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._


In [4]:
val spark = SparkSession.builder() 
  .master("local[*]") 
  .config("spark.executor.memory", "3g")
  .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse")
  .appName("NohupReader")
  .getOrCreate()


org.apache.spark.sql.SparkSession@3dfadb78

In [5]:
import spark.implicits._

val research_home: String = scala.util.Properties.envOrElse("RESEARCH_HOME", "/home/acald013/Research/")
val folder = s"${research_home}Scripts/Python/"
val prefix = "nohup"

val nohup = spark.read.textFile(s"${folder}${prefix}*")


org.apache.spark.sql.SparkSession$implicits$@45745a05

In [6]:
println(nohup.count)

22832


null

In [7]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class Line(line: String, n: Long)
case class Run(runID: Long, date: String, method: String, cores: Int, epsilon: Double, mu: Int, delta: Int, methodTime: Double)
case class Stage(runID: Long, n: Long, timestamp: String, stage: String, stageTime: Double, load: Int, unit: String)
case class Record(runID: Long, date: String, method: String, cores: Int, epsilon: Double, mu: Int, delta: Int, methodTime: Double, n: Long, timestamp: String, stage: String, stageTime: Double, load: Int, unit: String)

implicit class DatasetOps(ds: org.apache.spark.sql.Dataset[_]) {
    def display(rows: Int = 20) = {
        import com.twosigma.beakerx.scala.table.TableDisplay
        val columns = ds.columns
        val rowVals = ds.toDF.take(rows)
        val t = new TableDisplay(rowVals map (row => (columns zip row.toSeq).toMap))
        t.display()
    }
}

defined class Line
defined class Run
defined class Stage
defined class Record
defined class DatasetOps


In [8]:
val lines = nohup.toDF("line").withColumn("n", monotonicallyIncreasingId).as[Line].cache()
val nLines = lines.count()
lines.display(10)

null

In [13]:
val indicesRun = lines.filter{ l => 
        l.line.contains("=== MergeLast Start ===") || l.line.contains("method=MergeLast,") ||
        l.line.contains("=== SpatialJoin Start ===") || l.line.contains("method=SpatialJoin,")
    }
    .orderBy("n")
    .select("n")
    .collect()
    .toList
    .map(_.getLong(0))
    .grouped(2)
    .toList
    .map(pair => (pair.head, pair.last))
    .filter(r => r._1 != r._2)
    .zipWithIndex
val indexRun = spark.createDataset(indicesRun)
    .flatMap{ pair => 
        (pair._1._1 to pair._1._2)
        .toList.map(v => (pair._2, v))
    }
    .toDF("runID","n")
    .cache
indexRun.display(10)

null

In [14]:
val runs = indexRun.join(lines, "n").
    groupBy("runID").
    agg(max($"n").alias("n")).
    join(lines, "n").
    select("runID", "line").
    orderBy("runID").
    map{ row =>
        val runID = row.getInt(0)
        val line  = row.getString(1)
        var arr1  = line.split(" -> ")
        val date  = arr1(0)
        val arr2  = arr1(1).split(",")
        val method  = arr2(0).split("=")(1)
        val cores   = arr2(1).split("=")(1).toInt
        val epsilon = arr2(2).split("=")(1).toDouble
        val mu      = arr2(3).split("=")(1).toInt
        val delta   = arr2(4).split("=")(1).toInt
        val time    = arr2(5).split("=")(1).toDouble
        Run(runID, date, method, cores, epsilon, mu, delta, time)
    }.
    cache
runs.display(10)

null

In [15]:
val stages = lines.filter(_.line.contains("|")).
    join(indexRun, "n").
    map{ m =>
        val n         = m.getLong(0)
        val line      = m.getString(1)
        val runID     = m.getInt(2)
        var arr1      = line.split(" -> ")
        val timestamp = arr1(0).trim
        val arr2      = arr1(1).split("\\|")
        val stage     = arr2(0).trim
        val time      = arr2(1).trim.dropRight(1).toDouble
        val arr3      = arr2(2).trim.split(" ")
        val load      = arr3(0).toInt
        val unit      = arr3(1)
        Stage(runID, n, timestamp, stage, time, load, unit)
    }.cache
    
stages.count()
stages.display(10)

null

In [18]:
val ml_stages = stages.join(runs, "runID").
    filter($"method" === "MergeLast").
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage".alias("stage0"), $"stageTime").
    withColumn("stage1", regexp_replace($"stage0", "Reporting locations at t=\\d+", "0.Reporting locations")).
    withColumn("stage2", regexp_replace($"stage1", "Checking internal timestamps", "4.Checking internals")).
    withColumn("stage3", regexp_replace($"stage2", "\\.\\.\\.", "")).
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage3".alias("stage"), $"stageTime").
    filter(!$"stage".rlike("4.Distance Join phase")).
    filter(!$"stage".rlike("5.Getting candidates"))
ml_stages.show(truncate = false)
//ml_stages.display(20)


+-----+---+---------+-------+---+-----+----------------------------+---------+
|runID|n  |method   |epsilon|mu |delta|stage                       |stageTime|
+-----+---+---------+-------+---+-----+----------------------------+---------+
|0    |19 |MergeLast|10.0   |4  |5    |0.Reporting locations       |5.3      |
|0    |35 |MergeLast|10.0   |4  |5    |1.Set of disks for t_i      |29.4     |
|0    |36 |MergeLast|10.0   |4  |5    |0.Reporting locations       |5.01     |
|0    |52 |MergeLast|10.0   |4  |5    |2.Set of disks for t_i+delta|32.2     |
|0    |53 |MergeLast|10.0   |4  |5    |3.Joining timestams         |3.89     |
|0    |117|MergeLast|10.0   |4  |5    |4.Checking internals        |57.76    |
|0    |118|MergeLast|10.0   |4  |5    |0.Reporting locations       |4.76     |
|0    |134|MergeLast|10.0   |4  |5    |1.Set of disks for t_i      |18.14    |
|0    |135|MergeLast|10.0   |4  |5    |0.Reporting locations       |4.78     |
|0    |151|MergeLast|10.0   |4  |5    |2.Set of disk

null

In [20]:
val sj_stages = stages.join(runs, "runID").
    filter($"method" === "SpatialJoin").
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage".alias("stage0"), $"stageTime").
    withColumn("stage1", regexp_replace($"stage0", "\\.\\.\\.", "")).
    withColumn("stage2", regexp_replace($"stage1", "Reporting", "0.Reporting")).
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage2".alias("stage"), $"stageTime").
    filter(!$"stage".rlike("4.Distance Join phase")).
    filter(!$"stage".rlike("5.Getting candidates"))
sj_stages.show(truncate = false)
//sj_stages.display(20)


+-----+----+-----------+-------+---+-----+----------------------+---------+
|runID|n   |method     |epsilon|mu |delta|stage                 |stageTime|
+-----+----+-----------+-------+---+-----+----------------------+---------+
|5    |3325|SpatialJoin|10.0   |4  |5    |0.Reporting locations |27.57    |
|5    |3341|SpatialJoin|10.0   |4  |5    |1.Set of disks for t_i|141.36   |
|5    |3342|SpatialJoin|10.0   |4  |5    |4.Found flocks        |0.41     |
|5    |3343|SpatialJoin|10.0   |4  |5    |5.Updating times      |0.42     |
|5    |3344|SpatialJoin|10.0   |4  |5    |6.Filter phase        |0.86     |
|5    |3345|SpatialJoin|10.0   |4  |5    |0.Reporting locations |5.19     |
|5    |3361|SpatialJoin|10.0   |4  |5    |1.Set of disks for t_i|20.51    |
|5    |3362|SpatialJoin|10.0   |4  |5    |2.Distance Join phase |2.95     |
|5    |3363|SpatialJoin|10.0   |4  |5    |3.Getting candidates  |1.55     |
|5    |3364|SpatialJoin|10.0   |4  |5    |4.Found flocks        |0.2      |
|5    |3365|

null

In [ ]:
val data = runs.select($"runID", $"method", $"epsilon", $"mu", $"delta", $"methodTime".alias("time")).
    orderBy($"runID", $"epsilon", $"method").
    cache
data.count()
data.display(10)

In [ ]:
val d = data.collect.map(_.mkString(";")).mkString("\n")

import java.io._
val pw = new PrintWriter(new File(s"${folder}methods.csv" ))
pw.write(s"$d\n")
pw.close

In [21]:
val d = ml_stages.union(sj_stages).collect.map(_.mkString(";")).mkString("\n")

import java.io._
val pw = new PrintWriter(new File(s"${folder}stages.csv" ))
pw.write(s"$d\n")
pw.close

null